This notebook was inpired by [this LlamaIndex notebook](https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/agent/coa_agent.ipynb)

[Original Chain-of-Abstraction paper](https://arxiv.org/abs/2401.17464)

Making some changes to it with the only intention of trying ideas and learning.

Notice that I am assuming you have the relevant API_KEYs as environmental variables.

In [4]:
%pip install llama-index-packs-agents-coa

In [12]:
from bubls.utils.data.download import download_file_from_url
from bubls.utils.indexing import create_index_from_path
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.packs.agents_coa import CoAAgentWorker
import os
import nest_asyncio
nest_asyncio.apply()

## LlamaIndex Settings

In [4]:
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small", embed_batch_size=256
)
Settings.llm = OpenAI(model="gpt-4-turbo", temperature=0.1)


## Defining global variables

In [5]:
METADATA = {
    "lyft_10k": {
        "source_url": "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf",
        "file_name": "lyft_10k_2021.pdf",
        "save_data_to": os.path.join(os.environ["DATA_DIR"], "lyft_10k"),
        "persist_index_to": os.path.join(os.environ["PERSIST_DIR"], "lyft_10k"),
        },
    "uber_10k": {
        "source_url": "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf",
        "file_name": "uber_10k_2021.pdf",
        "save_data_to": os.path.join(os.environ["DATA_DIR"], "uber_10k"),
        "persist_index_to": os.path.join(os.environ["PERSIST_DIR"], "uber_10k"),
        },
}

## Ingest Data
- Download Information
- Create&Persist or Load Index 
- Create Query Engine

In [6]:
# parser = LlamaParse(result_type="markdown")

In [8]:
query_engine_dict = {}
for k, md in METADATA.items():
    download_file_from_url(md["source_url"], md["file_name"], md["save_data_to"])
    index = create_index_from_path(
        md["persist_index_to"],
        md["save_data_to"],
        # {".pdf": parser}
    )
    query_engine_dict[k] = index.as_query_engine(similarity_top_k=3)

Loading Index
Loading Index


## Define tool

In [9]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine_dict[k],
        metadata=ToolMetadata(
            name=k,
            description=(
                f"Provides information about {k} financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    )
    for k in METADATA.keys()
]

## Agent

Steps
- The tools get parsed into python-like definitions
- The agent is prompted to generate a CoA plan
- The function calls are parsed out of the plan and executed
- The values in the plan are filled in
- The agent generates a final response

In [13]:
worker = CoAAgentWorker.from_tools(
    tools=query_engine_tools,
    llm=Settings.llm,
    verbose=True,
)
agent = AgentRunner(worker)
response = agent.chat("How did Ubers revenue growth compare to Lyfts in 2021?")

==== Available Parsed Functions ====
def lyft_10k(input: string):
   """Provides information about lyft_10k financials for year 2021. Use a detailed plain text question as input to the tool."""
    ...
def uber_10k(input: string):
   """Provides information about uber_10k financials for year 2021. Use a detailed plain text question as input to the tool."""
    ...
==== Generated Chain of Abstraction ====
Abstract plan of reasoning:
1. Retrieve Uber's revenue for 2021 by querying the Uber financial tool with a specific question about revenue. This can be done using the function call [FUNC uber_10k("What was Uber's revenue in 2021?") = y1].
2. Retrieve Lyft's revenue for 2021 by querying the Lyft financial tool with a similar question about revenue. This can be done using the function call [FUNC lyft_10k("What was Lyft's revenue in 2021?") = y2].
3. Compare the revenue figures obtained from Uber and Lyft to determine which company had greater revenue growth. This comparison will be based

In [14]:
print(str(response))

The provided context information does not include specific details about the revenue figures for Uber and Lyft in 2021. Therefore, I am unable to determine how Uber's revenue growth compared to Lyft's in that year.


## How the prompting of Chain of Abstraction works

How does this actually work?

So, under the hood we are prompting the LLM to first output the CoA, then we parse it and run functions, then we refine all that into a final output.

First, we parse the tools into python-like function defintions by parsing `tool.metadata.fn_schema_str`, along with the tool name and description.

You can find that code in the [utils](https://notebooks.githubusercontent.com/view/ipynb?browser=firefox&bypass_fastly=true&color_mode=auto&commit=7b52057b717451a801c583fae7efe4c4ad167455&device=unknown_device&docs_host=https%3A%2F%2Fdocs.github.com&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f72756e2d6c6c616d612f6c6c616d615f696e6465782f376235323035376237313734353161383031633538336661653765666534633461643136373435352f646f63732f646f63732f6578616d706c65732f6167656e742f636f615f6167656e742e6970796e62&logged_in=true&nwo=run-llama%2Fllama_index&path=docs%2Fdocs%2Fexamples%2Fagent%2Fcoa_agent.ipynb&platform=linux&repository_id=560704231&repository_type=Repository&version=125).

What this looks like is we have a prompt like this:

REASONING_PROMPT_TEMPALTE = """Generate an abstract plan of reasoning using placeholders for the specific values and function calls needed.
The placeholders should be labeled y1, y2, etc.
Function calls should be represented as inline strings like [FUNC {{function_name}}({{input1}}, {{input2}}, ...) = {{output_placeholder}}].
Assume someone will read the plan after the functions have been executed in order to make a final response.
Not every question will require function calls to answer.
If you do invoke a function, only use the available functions, do not make up functions.

Example:
-----------
Available functions:
\`\`\`python
def add(a: int, b: int) -> int:
    \"\"\"Add two numbers together.\"\"\"
    ...

def multiply(a: int, b: int) -> int:
    \"\"\"Multiply two numbers together.\"\"\"
    ...
\`\`\`

Question:
Sally has 3 apples and buys 2 more. Then magically, a wizard casts a spell that multiplies the number of apples by 3. How many apples does Sally have now?

Abstract plan of reasoning:
After buying the apples, Sally has [FUNC add(3, 2) = y1] apples. Then, the wizard casts a spell to multiply the number of apples by 3, resulting in [FUNC multiply(y1, 3) = y2] apples.

Your Turn:
-----------
Available functions:
\`\`\`python
{functions}
\`\`\`

Question:
{question}

Abstract plan of reasoning:
"""

This will generate the chain-of-abstraction reasoning.

Then, the reasoning is parsed using the [output parser](https://notebooks.githubusercontent.com/view/ipynb?browser=firefox&bypass_fastly=true&color_mode=auto&commit=7b52057b717451a801c583fae7efe4c4ad167455&device=unknown_device&docs_host=https%3A%2F%2Fdocs.github.com&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f72756e2d6c6c616d612f6c6c616d615f696e6465782f376235323035376237313734353161383031633538336661653765666534633461643136373435352f646f63732f646f63732f6578616d706c65732f6167656e742f636f615f6167656e742e6970796e62&logged_in=true&nwo=run-llama%2Fllama_index&path=docs%2Fdocs%2Fexamples%2Fagent%2Fcoa_agent.ipynb&platform=linux&repository_id=560704231&repository_type=Repository&version=125).

After calling the functions and filling in values, we give the LLM a chance to refine the response, using this prompt:

REFINE_REASONING_PROMPT_TEMPALTE = """Generate a response to a question by using a previous abstract plan of reasoning. Use the previous reasoning as context to write a response to the question.

Example:
-----------
Question: 
Sally has 3 apples and buys 2 more. Then magically, a wizard casts a spell that multiplies the number of apples by 3. How many apples does Sally have now?

Previous reasoning:
After buying the apples, Sally has [FUNC add(3, 2) = 5] apples. Then, the wizard casts a spell to multiply the number of apples by 3, resulting in [FUNC multiply(5, 3) = 15] apples.

Response:
After the wizard casts the spell, Sally has 15 apples.

Your Turn:
-----------
Question:
{question}

Previous reasoning:
{prev_reasoning}

Response:
"""

